In [1]:
# Some standard imports
import io
import numpy as np
import torch
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

/home/mazheng/miniconda3/envs/eff/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys 
sys.path.append("..")
from backbone import EfficientDetBackbone
from utils.vis_utils import model_params, model_load
from utils.eff_utils import load_yaml
from dataclasses import dataclass
from utils.utils import preprocess

In [3]:
from pathlib import Path
path = Path.cwd().parent

In [4]:
project_name = "0509split"
# project_name = "4-4"
# project_name = "detection-untunnel"
compound_coef = 2
threshold = 0.3
iou_threshold = 0.2
# model_path = 'logs/0509split/efficientdet-d2_71_145500.pth'
model_path = str(path/'logs/0509split_d2/efficientdet-d2_125_253500.pth')

In [5]:
ann_json = path / 'datasets/{}/annotations/instances_val.json'.format(project_name)
# ann_json = path / 'datasets/val_dataset/0409/0409notunnel/instance_val.json'
img_path = path / 'datasets/{}/val'.format(project_name)
# img_path = Path('datasets/val_dataset/0409/0409notunnel/img')
# img_path = Path('datasets/4-4')
yaml_path = path / 'projects/{}.yml'.format(project_name)
# yaml_path = path / 'projects/{}_3anchor.yml'.format(project_name)
# yaml_path = path / 'projects/0414split.yml'

project_params = load_yaml(str(yaml_path))

ratios=eval(project_params['anchors_ratios'])
scales=eval(project_params['anchors_scales'])
# scales=[(2 ** 0)/2, (2 ** (1.0 / 3.0))/2, (2 ** (2.0 / 3.0))/2]
# ratios=[(0.6, 1.6), (1.2, 0.8), (1.7, 0.6)]
obj_list = project_params['obj_list']
params = model_params(compound_coef, obj_list, ratios, scales, model_path)

In [6]:
params

model_params(compound_coef=2, obj_list=['pounding', 'pothole', 'hcrack', 'rsign', 'vcrack', 'animal', 'csign', 'people', 'indicator-red', 'lcrack', 'spiledmaterial', 'tsign', 'gantry', 'osign', 'bsign', 'label', 'fracturing', 'indicator-green', 'indicator-off', 'light-off'], ratios=[(0.7, 1.5), (0.9, 1.1), (1.2, 0.8), (1.7, 0.6), (3.3, 0.3)], scales=[0.35714285714285715, 0.44997180353388333, 0.5669289471314998], model_path='/home/mazheng/waste-detection/Yet-Another-EfficientDet-Pytorch/logs/0509split_d2/efficientdet-d2_125_253500.pth')

In [9]:
model = EfficientDetBackbone(
    onnx_export=True,
    compound_coef=params.compound_coef,
    num_classes=len(params.obj_list),
    # replace this part with your project's anchor config
    ratios=params.ratios,
    scales=params.scales,
)
# model.backbone_net.model.set_swish(memory_efficient=False)
model.load_state_dict(torch.load(params.model_path))
model.eval()

EfficientDetBackbone(
  (bifpn): Sequential(
    (0): BiFPN(
      (conv6_up): SeparableConvBlock(
        (depthwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(112, 112, kernel_size=(3, 3), stride=(1, 1), groups=112, bias=False)
        )
        (pointwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(112, 112, kernel_size=(1, 1), stride=(1, 1))
        )
        (bn): BatchNorm2d(112, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
      (conv5_up): SeparableConvBlock(
        (depthwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(112, 112, kernel_size=(3, 3), stride=(1, 1), groups=112, bias=False)
        )
        (pointwise_conv): Conv2dStaticSamePadding(
          (conv): Conv2d(112, 112, kernel_size=(1, 1), stride=(1, 1))
        )
        (bn): BatchNorm2d(112, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
      (conv4_up): SeparableConvBlock(
        (depthwise_conv): Conv2dStaticSamePad

In [7]:
img_name = "DLLJ1796.jpg"
str(img_path/img_name)

'/home/mazheng/waste-detection/Yet-Another-EfficientDet-Pytorch/datasets/0509split/val/DLLJ1796.jpg'

In [28]:
input_sizes=[512, 640, 768, 896, 1024, 1280, 1280, 1536]
input_size = (
        input_sizes[model.compound_coef]
    )

ori_imgs, framed_imgs, framed_metas = preprocess(
    str(img_path/img_name),
    max_size=input_size,
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225],
)
x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)
x = x.permute(0, 3, 1, 2)

In [29]:
_x = np.stack(framed_imgs, 0)
_x = np.moveaxis(_x,[0, 3, 1, 2],[0,1,2,3])

In [30]:
_x.shape
_x = torch.from_numpy(_x)

In [31]:
torch.allclose(x,_x)

True

In [15]:
print(framed_imgs[0].shape)

(768, 768, 3)


In [35]:
batch_size = 1
x = torch.randn(batch_size, 3, 768, 768, requires_grad=True)
# features, regression, classification, anchors = model(x)
# print([i.shape for i in torch_out])
# Export the model


In [36]:
torch.onnx.export(model, x,
                  'efficientdet-d2.onnx',
                  verbose=False,
                  input_names=['data'],
                  opset_version=11)

In [ ]:
device = torch.device('cuda')
model = EfficientDetBackbone(
        onnx_export=True,
        compound_coef=params.compound_coef,
        num_classes=len(params.obj_list),
        # replace this part with your project's anchor config
        ratios=params.ratios,
        scales=params.scales,
    )

model.backbone_net.model.set_swish(memory_efficient=False)

# dummy_input = torch.randn((1,3,768, 768), dtype=torch.float32).to(device)

model.load_state_dict(torch.load(params.model_path))
model =model.eval()
# model = model.cuda()
# opset_version can be changed to 10 or other number, based on your need
torch.onnx.export(model, x,
                  'efficientdet-d0.onnx',
                  verbose=False,
                  input_names=['data'],
                  opset_version=11)

In [37]:
import onnx

# Load the ONNX model
model_ = onnx.load("efficientdet-d2.onnx")

# Check that the model is well formed
onnx.checker.check_model(model_)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model_.graph))

graph torch-jit-export (
  %data[FLOAT, 1x3x768x768]
) initializers (
  %backbone_net.model._blocks.0._bn1.bias[FLOAT, 32]
  %backbone_net.model._blocks.0._bn1.num_batches_tracked[INT64, scalar]
  %backbone_net.model._blocks.0._bn1.running_mean[FLOAT, 32]
  %backbone_net.model._blocks.0._bn1.running_var[FLOAT, 32]
  %backbone_net.model._blocks.0._bn1.weight[FLOAT, 32]
  %backbone_net.model._blocks.0._bn2.bias[FLOAT, 16]
  %backbone_net.model._blocks.0._bn2.num_batches_tracked[INT64, scalar]
  %backbone_net.model._blocks.0._bn2.running_mean[FLOAT, 16]
  %backbone_net.model._blocks.0._bn2.running_var[FLOAT, 16]
  %backbone_net.model._blocks.0._bn2.weight[FLOAT, 16]
  %backbone_net.model._blocks.0._depthwise_conv.conv.weight[FLOAT, 32x1x3x3]
  %backbone_net.model._blocks.0._project_conv.conv.weight[FLOAT, 16x32x1x1]
  %backbone_net.model._blocks.0._se_expand.conv.bias[FLOAT, 32]
  %backbone_net.model._blocks.0._se_expand.conv.weight[FLOAT, 32x8x1x1]
  %backbone_net.model._blocks.0._se_red

In [38]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("efficientdet-d2.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

2022-05-13 20:45:03.163281918 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer 'backbone_net.model._blocks.16._bn2.num_batches_tracked'. It is not used by any node and should be removed from the model.
2022-05-13 20:45:03.163351334 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer 'classifier.bn_list.0.1.num_batches_tracked'. It is not used by any node and should be removed from the model.
2022-05-13 20:45:03.163377990 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer 'bifpn.4.conv3_up.bn.num_batches_tracked'. It is not used by any node and should be removed from the model.
2022-05-13 20:45:03.163447691 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer 'backbone_net.model._blocks.0._bn2.num_batches_tracked'. It is not used by any node and should be removed from the model.
2022-05-13 20:45:03.163476737 [W:onnxruntime:, graph.cc:3559 Cle

In [39]:
features, regression, classification, anchors = model(x)

In [42]:
# features, regression, classification, anchors
np.testing.assert_allclose(to_numpy(features[3]), ort_outs[3], rtol=1e-03, atol=1e-05)

AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

Mismatched elements: 85 / 16128 (0.527%)
Max absolute difference: 9.1552734e-05
Max relative difference: 0.33803645
 x: array([[[[-0.608884, -0.612374, -0.663911, ..., -0.746684, -0.595294,
          -0.693546],
         [-0.441964, -0.688357, -0.466391, ..., -0.368052, -0.530193,...
 y: array([[[[-0.608881, -0.612376, -0.663905, ..., -0.746674, -0.595288,
          -0.693544],
         [-0.441964, -0.688353, -0.466386, ..., -0.36804 , -0.530201,...